In [1]:


import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))


def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations
                               
                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)
    
    text=emoji_pattern.sub(r'', text)
    text=english_pattern.sub(r'', text)

    return text

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰⚽️✌�￰৷￰'''
    
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out

2023-09-02 14:53:58.495187: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-02 14:53:58.526607: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-02 14:53:58.527287: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-02 14:53:59.093107: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:


train_url = 'https://raw.githubusercontent.com/SyedT1/NLP/main/FromScratch/error%20detection/data/data/train_data.csv'
test_url = 'https://raw.githubusercontent.com/SyedT1/NLP/main/FromScratch/error%20detection/data/data/test_data.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('data/stopwords_bangla.xlsx',index_col=False)
STOPWORDS = set([word.strip() for word in stop_words_df['words']])



In [3]:
df_train['Comment'] = df_train.Comment.apply(lambda x: preprocessing(str(x)))
df_test['Comment'] = df_test.Comment.apply(lambda x:preprocessing(str(x)))
df = pd.concat([df_train,df_test],ignore_index = True)



In [4]:
def encode(s):
    d = {
        "Code Switching":0,
        "Grammatical":1,
        "Multiple Errors":2,
        "Spelling":3
    }
    if s in d:
        return d[s]
    else:
        return 4
df['Category'] = df.Category.apply(lambda x: encode(x))
df_train['Category'] = df_train.Category.apply(lambda x: encode(x))
df_test['Category'] = df_test.Category.apply(lambda x: encode(x))


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Attention
from tensorflow.keras.models import Model
# import tensorflow_addons as tfa

# Sample data and labels (replace with your own dataset)
data = df_train['Comment']
labels = df_train['Category']

# Tokenize text data
max_words = 1000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
word_index = tokenizer.word_index
max_sequence_length = 100  # Adjust this based on your data

# Pad sequences
X = pad_sequences(sequences, maxlen=max_sequence_length)

# One-hot encode labels
Y = tf.keras.utils.to_categorical(labels, num_classes=5)

# Create LSTM with Attention model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(max_words, 100)(input_layer)
lstm_layer = LSTM(64, return_sequences=True)(embedding_layer)

# Attention mechanism using TensorFlow Addons
attention = Attention()([lstm_layer, lstm_layer])
attention = tf.keras.layers.GlobalAveragePooling1D()(attention)

output_layer = Dense(5, activation='sigmoid')(attention)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, Y, epochs=10, batch_size=16, validation_split=0.2)

# Evaluate the model on test data (replace with your test data)
test_data = df_test['Comment'] # Replace with your actual test data
test_labels = df_test['Category']  # Replace with your actual test labels

test_sequences = tokenizer.texts_to_sequences(test_data)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_length)
Y_test = tf.keras.utils.to_categorical(test_labels, num_classes=5)

loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10


2023-09-02 14:54:04.538547: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "101" frequency: 3600 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 6291456 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


400/402 [============================>.] - ETA: 0s - loss: 1.1788 - accuracy: 0.5683

2023-09-02 14:54:12.752845: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "101" frequency: 3600 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 6291456 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


402/402 [==============================] - 10s 21ms/step - loss: 1.1787 - accuracy: 0.5683 - val_loss: 1.1426 - val_accuracy: 0.5883
Epoch 2/10
402/402 [==============================] - 8s 20ms/step - loss: 1.1220 - accuracy: 0.5846 - val_loss: 1.0070 - val_accuracy: 0.6244
Epoch 3/10
402/402 [==============================] - 8s 20ms/step - loss: 0.9481 - accuracy: 0.6535 - val_loss: 0.9472 - val_accuracy: 0.6561
Epoch 4/10
402/402 [==============================] - 8s 21ms/step - loss: 0.8674 - accuracy: 0.6819 - val_loss: 0.9513 - val_accuracy: 0.6468
Epoch 5/10
402/402 [==============================] - 9s 22ms/step - loss: 0.8095 - accuracy: 0.6990 - val_loss: 1.0359 - val_accuracy: 0.5933
Epoch 6/10
402/402 [==============================] - 8s 21ms/step - loss: 0.7631 - accuracy: 0.7133 - val_loss: 1.0215 - val_accuracy: 0.6238
Epoch 7/10
402/402 [==============================] - 8s 20ms/step - loss: 0.7170 - accuracy: 0.7232 - val_loss: 1.0287 - val_accuracy: 0.6381
Epoch 8/1

In [7]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
y_pred = model.predict(X_test)

# Calculate macro recall
macro_recall = recall_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
macro_precision = precision_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
# Print the results
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Macro Recall: {:0.3f}\n  Macro Precision: {:0.3f}'.format(loss, accuracy, macro_recall, macro_precision))


13/63 [=====>........................] - ETA: 0s

2023-09-02 14:56:13.033590: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "101" frequency: 3600 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 6291456 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


63/63 [==============================] - 1s 9ms/step
Test set
  Loss: 1.211
  Accuracy: 0.594
  Macro Recall: 0.392
  Macro Precision: 0.440
